In [ ]:
import requests
import json
import pandas as pd
import re
import time
import os
import random
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import nltk
from nltk.tokenize import word_tokenize
import csv

# Set up necessary directories and configurations:
os.makedirs('data', exist_ok=True)
session = requests.Session()
retries = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))
session.mount('https://', HTTPAdapter(max_retries=retries))
nltk.download('punkt')

# Clean title by standardizing the "By H. P. Lovecraft" text:
def clean_title(title):
    author_text = "By H. P. Lovecraft"
    title = re.sub(rf"({author_text}\s*)+", author_text, title).strip()
    if title.endswith(author_text) and not title.endswith(" " + author_text):
        title = title.replace(author_text, " " + author_text)
    return title

# --- Step 1: Scraping Lovecraft Fiction Works ---

def scrape_lovecraft_content():
    base_url = "https://www.hplovecraft.com/writings/texts/"
    response = session.get(base_url)
    
    if response.status_code != 200:
        print(f"Failed to access the base URL: {response.status_code}")
        return
    
    soup = BeautifulSoup(response.content, 'html.parser')
    content_links = [
        f"{base_url}{link['href']}"
        for link in soup.find_all('a', href=True)
        if link['href'].startswith('fiction/') and not link['href'].startswith('#')
    ]

    csv_filename = 'data/lovecraft_fiction.csv'
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Content Type', 'Title', 'Text'])

        for content_url in content_links:
            time.sleep(random.uniform(1, 3))
            try:
                content_response = session.get(content_url, headers={'User-Agent': 'Mozilla/5.0'})
                if content_response.status_code == 200:
                    content_soup = BeautifulSoup(content_response.content, 'html.parser')
                    title_tag = content_soup.find('font', size="+2")
                    author_tag = content_soup.find('font', size="+1")
                    text_div = content_soup.find('div', align='justify')

                    if title_tag and text_div:
                        title = f"{title_tag.get_text(strip=True)} by {author_tag.get_text(strip=True)}"
                        title = clean_title(title)  # Clean the title text
                        csvwriter.writerow(["fiction", title, text_div.get_text(strip=True)])
                        print(f'Scraped: {title}')
                    else:
                        print(f'Title or text not found for {content_url}')
                else:
                    print(f'Failed to scrape {content_url}: {content_response.status_code}')
            except Exception as e:
                print(f'Error scraping {content_url}: {e}')

# Scrape only fiction content:
scrape_lovecraft_content()


In [ ]:
# Load the scraped CSV data into a Pandas DataFrame
df = pd.read_csv('data/lovecraft_fiction.csv')

# View the first few rows of the dataframe to verify
print(df.head())


In [ ]:
from nltk.corpus import stopwords

# Ensure stopwords are downloaded
nltk.download('stopwords')

# Tokenize and remove stopwords
def clean_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    return " ".join(cleaned_tokens)

# Apply to the text column in the dataframe
df['Cleaned_Text'] = df['Text'].apply(clean_text)


In [ ]:
from collections import Counter

# Create a frequency distribution for words in all texts
all_words = " ".join(df['Cleaned_Text']).split()
word_freq = Counter(all_words)

# Display the most common words
print(word_freq.most_common(10))


In [ ]:
from wordcloud import WordCloud

# Create a word cloud from the cleaned text
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(" ".join(df['Cleaned_Text']))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


In [ ]:
import spacy
from collections import Counter

# Load the spaCy model for Named Entity Recognition
nlp = spacy.load("en_core_web_sm")

# Function to extract entities
def extract_entities(text):
    doc = nlp(text)
    entities = [ent.text.lower() for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE', 'LOC']]
    return entities

# Apply the extraction to the 'Cleaned_Text' column
df['Entities'] = df['Cleaned_Text'].apply(extract_entities)

# Flatten the list of entities and get their frequency count
all_entities = [entity for sublist in df['Entities'] for entity in sublist]
entity_freq = Counter(all_entities)

# Display the most common entities
print(entity_freq.most_common(100))


In [ ]:
# Custom list of Lovecraft entities to track
lovecraft_entities = [
    "cthulhu", "yog-sothoth", "nyarlathotep", "innsmouth", "arkham", "dunwich", "azathoth", "shub-niggurath",
    "hastur", "miskatonic", "the king in yellow", "the dark young", "the colour out of space", "the great old ones"
]

# Extract frequencies for these specific entities
specific_entity_freq = {entity: all_entities.count(entity) for entity in lovecraft_entities}

# Display the count of these specific entities
print(specific_entity_freq)


In [ ]:
import spacy
import re
from collections import Counter

# Load the spaCy model for Named Entity Recognition
nlp = spacy.load("en_core_web_sm")

# Expanded list of Lovecraftian and related entities (including new ones)
lovecraft_entities_expanded = [
    "cthulhu", "yog-sothoth", "nyarlathotep", "azathoth", "hastur", "r'lyeh", "dagon", 
    "shub-niggurath", "the great old ones", "elder gods", "the old ones", "the deep ones", "night gaunts", 
    "cthulhu cult", "the nameless city", "the black stone", "the dreamlands", "fenric", "hecuba", 
    "animus", "tor-gasukk", "moloch", "kai'lizakia", "lloigor", "eidolon", "derleth", "gog", "magog", "to'koth", 
    "karnas'koi", "traguam", "archon", "mi'en kalarash", "kwundaar", "volund", "k'thun", "noth-yidik", "tru'nembra", 
    "tulzscha", "cxaxukluth", "d'endrrah", "ubbo-sathla", "xexanoth", "ycnàgnnisssz", "yhoundeh", "aiueb gnshal", 
    "aletheia", "azhorra-tha", "c'thalpa", "daoloth", "ghroth", "gi-hoveg", "haiogh-yai", "huitloxopetl", "ialdagorth", 
    "kaajh'kaalbh", "kaalut", "lu-kthu", "mh'ithrha", "mlandoth", "mril thorion", "mother of pus", "nhimbaloth", 
    "ngyr-khorath", "nyctelios", "olkoth", "ramasekva", "shabbith-ka", "star mother", "suc'naath", "uvhash", "xa'ligha", 
    "yibb-tstll", "yidhra", "yomag'n'tho"
]

# Function to extract entities
def extract_entities(text):
    doc = nlp(text)
    
    # Initialize the list to store extracted entities
    entities = []
    
    # Extract the named entities using spaCy's NER (PERSON, ORG, GPE, LOC)
    for ent in doc.ents:
        if ent.label_ in ['PERSON', 'ORG', 'GPE', 'LOC']:
            entities.append(ent.text.lower())
    
    # Manually add expanded entities and regex pattern matches for indirect references
    for entity in lovecraft_entities_expanded:
        # Check for direct entity mentions (singular and plural)
        singular_entity = r'\b' + re.escape(entity) + r'\b'
        plural_entity = r'\b' + re.escape(entity + "s") + r'\b'  # Handle plural form

        if re.search(singular_entity, text.lower()) or re.search(plural_entity, text.lower()):
            entities.append(entity)
        
        # Check for indirect references or variations (e.g., "great old ones" or "eldritch horror")
        indirect_references = [
            r"\bdeep ones\b", r"\bcosmic entity\b", r"\bhorrible being\b", r"\bnight gaunts\b", r"\bblack stone\b",
            r"\byog sothoth\b", r"\bnamesless city\b", r"\bstrange entity\b", r"\botherworldly creature\b", r"\bdark god\b",
            r"\bhorrible power\b", r"\btimeless one\b"
        ]
        for pattern in indirect_references:
            if re.search(pattern, text.lower()):
                entities.append(entity)
    
    return entities

# Apply the extraction to the 'Cleaned_Text' column (assuming 'Cleaned_Text' contains the content)
df['Entities'] = df['Cleaned_Text'].apply(extract_entities)

# Flatten the list of entities and get their frequency count
all_entities = [entity for sublist in df['Entities'] for entity in sublist]
entity_freq = Counter(all_entities)

# Display the most common entities
print(entity_freq.most_common(100))

# Custom list of Lovecraft entities to track (including new entities)
specific_entity_freq = {entity: all_entities.count(entity) for entity in lovecraft_entities_expanded}

# Display the count of these specific entities
print(specific_entity_freq)


In [ ]:
import spacy
import re
from collections import Counter

# Load the spaCy model for Named Entity Recognition
nlp = spacy.load("en_core_web_sm")

# Expanded list of Lovecraftian and related entities (including new ones)
lovecraft_entities_expanded = [
    "cthulhu", "yog-sothoth", "nyarlathotep", "azathoth", "hastur", "r'lyeh", "dagon", 
    "shub-niggurath", "the great old ones", "elder gods", "the old ones", "the deep ones", "night gaunts", 
    "cthulhu cult", "the nameless city", "the black stone", "the dreamlands", "fenric", "hecuba", 
    "animus", "tor-gasukk", "moloch", "kai'lizakia", "lloigor", "eidolon", "derleth", "gog", "magog", "to'koth", 
    "karnas'koi", "traguam", "archon", "mi'en kalarash", "kwundaar", "volund", "k'thun", "noth-yidik", "tru'nembra", 
    "tulzscha", "cxaxukluth", "d'endrrah", "ubbo-sathla", "xexanoth", "ycnàgnnisssz", "yhoundeh", "aiueb gnshal", 
    "aletheia", "azhorra-tha", "c'thalpa", "daoloth", "ghroth", "gi-hoveg", "haiogh-yai", "huitloxopetl", "ialdagorth", 
    "kaajh'kaalbh", "kaalut", "lu-kthu", "mh'ithrha", "mlandoth", "mril thorion", "mother of pus", "nhimbaloth", 
    "ngyr-khorath", "nyctelios", "olkoth", "ramasekva", "shabbith-ka", "star mother", "suc'naath", "uvhash", "xa'ligha", 
    "yibb-tstll", "yidhra", "yomag'n'tho"
]

# Function to extract entities
def extract_entities(text):
    doc = nlp(text)
    
    # Initialize the list to store extracted entities
    entities = []
    
    # Manually add expanded entities and regex pattern matches for direct references
    for entity in lovecraft_entities_expanded:
        # Check for direct entity mentions (singular and plural)
        singular_entity = r'\b' + re.escape(entity) + r'\b'
        plural_entity = r'\b' + re.escape(entity + "s") + r'\b'  # Handle plural form

        if re.search(singular_entity, text.lower()) or re.search(plural_entity, text.lower()):
            entities.append(entity)
        
        # Check for indirect references or variations (e.g., "great old ones" or "eldritch horror")
        indirect_references = [
            r"\bdeep ones\b", r"\bcosmic entity\b", r"\bhorrible being\b", r"\bnight gaunts\b", r"\bblack stone\b",
            r"\byog sothoth\b", r"\bnamesless city\b", r"\bstrange entity\b", r"\botherworldly creature\b", r"\bdark god\b",
            r"\bhorrible power\b", r"\btimeless one\b"
        ]
        for pattern in indirect_references:
            if re.search(pattern, text.lower()):
                entities.append(entity)
    
    return entities

# Apply the extraction to the 'Cleaned_Text' column (assuming 'Cleaned_Text' contains the content)
df['Entities'] = df['Cleaned_Text'].apply(extract_entities)

# Flatten the list of entities and get their frequency count
all_entities = [entity for sublist in df['Entities'] for entity in sublist]
entity_freq = Counter(all_entities)

# Display the most common entities
print(entity_freq.most_common(100))

# Custom list of Lovecraft entities to track (including new entities)
specific_entity_freq = {entity: all_entities.count(entity) for entity in lovecraft_entities_expanded}

# Display the count of these specific entities
print(specific_entity_freq)


In [51]:
# List of entities with their mentions
entities = [
     ('carter', 175), ('willett', 99), ('arkham', 74), ('joseph curwen', 53),
    ('boston', 51), ('whateley', 45), ('nyarlathotep', 42), ('clarendon', 35),
    ('london', 33), ('new york', 32), ('miskatonic', 27), ('africa', 26),
    ('cthulhu', 22), ('vermont', 21), ('egypt', 20), ('johnny', 20),
    ('paris', 19), ('ammi', 19), ('denis', 19), ('great cthulhu', 18),
    ('innsmouth', 18), ('azathoth', 18), ('eidolon', 18), ('europe', 17),
    ('elder gods', 17), ('dagon', 17), ('mummy', 17), ('san francisco', 16),
    ('washington', 16), ('thou', 16), ('wolf', 15), ('cthulhu cult', 15),
    ('dunwich', 14), ('hastur', 14), ('dunwich horror', 14), ('moloch', 14),
    ('derleth', 14), ('archon', 14), ('cairo', 14), ('grey', 14), ('wilbur', 14),
    ('ben', 14), ('dobson', 14), ('yog-sothoth', 13), ('shub-niggurath', 13),
    ('the king in yellow', 13), ('the dark young', 13), ('colour out of space', 13),
    ('the great old ones', 13), ('the old ones', 13), ('the deep ones', 13),
    ('night gaunts', 13), ('the nameless city', 13), ('the black stone', 13),
    ('the dreamlands', 13), ("r'lyeh", 13), ('yog sothoth', 13),
    ('the whisperer in darkness', 13), ('the colour out of space', 13),
    ('the shadow over innsmouth', 13), ('the great intelligence', 13),
    ('fenric', 13), ('nestene consciousness', 13), ('guardians of time', 13),
    ('celestial toymaker', 13), ('gods of ragnarok', 13), ('hecuba', 13),
    ('animus', 13), ('tor-gasukk', 13), ("kai'lizakia", 13), ('lloigor', 13),
    ('toymakers', 13), ('gog and magog', 13), ("to'koth", 13), ("karnas'koi", 13),
    ('traguam', 13), ("mi'en kalarash", 13), ('kwundaar', 13), ('volund', 13),
    ("k'thun", 13), ('the nameless mist', 13), ('noth-yidik', 13), ("tru'nembra", 13),
    ('tulzscha', 13), ('abhoth', 13), ('cxaxukluth', 13), ("d'endrrah", 13),
    ('ubbo-sathla', 13), ('xexanoth', 13), ('ycnàgnnisssz', 13), ('yhoundeh', 13),
    ('aiueb gnshal', 13), ('aletheia', 13), ('azhorra-tha', 13),
    ('the blackness from the stars', 13), ('the cloud-thing', 13), ("c'thalpa", 13),
    ('daoloth', 13), ('ghroth', 13), ('gi-hoveg', 13), ('haiogh-yai', 13),
    ('huitloxopetl', 13), ('ialdagorth', 13), ("kaajh'kaalbh", 13), ('kaalut', 13),
    ('lu-kthu', 13), ("mh'ithrha", 13), ('mlandoth', 13), ('mril thorion', 13),
    ('mother of pus', 13), ('nhimbaloth', 13), ('ngyr-khorath', 13), ('nyctelios', 13),
    ('olkoth', 13), ('ramasekva', 13), ('shabbith-ka', 13), ('star mother', 13),
    ("suc'naath", 13), ('uvhash', 13), ("xa'ligha", 13), ('yibb-tstll', 13),
    ('yidhra', 13), ("yomagn'tho", 13), ('california', 13), ('mexico city', 13),
    ('george campbell', 12), ('fed', 12), ('van der', 12), ('france', 11),
    ('us', 11), ('new orleans', 11), ('john brown', 11), ('randolph carter', 11),
    ('steve', 11), ('dalton', 11), ('joe slater', 10), ('america', 10),
    ('nahum', 10), ('moon', 10), ('davis', 10), ('wilbur whateley', 10),
    ('mexico', 10), ('de marigny', 10), ('rome', 9), ('gulf', 9),
    ('house olney court', 9), ('ezra weeden', 9), ('spain', 9), ('zoogs', 9),
    ('san quentin', 9), ('china', 9), ('kodak', 9), ('williams', 8), ('warren', 8),
    ('titan', 8), ('crest', 8), ('joe mazurewicz', 8), ('florida', 8),
    ('john', 8), ('nova', 8), ('romero', 8), ('robert grandison', 8), ('vista', 7),
    ('galley', 7), ('massachusetts', 7), ('joe', 7), ('arthur jermyn', 7),
    ('jermyn house', 7), ('virginia', 7), ('nile', 7), ('van keulen', 7),
    ('india', 6), ('rhode island', 6), ('arkansas', 6), ('pinnacle', 6),
    ('keziah', 6), ('sacramento', 6), ('chicago', 6), ('bell', 6), ('robert suydam', 6),
    ('sepulchre', 6), ('anderson', 6), ('james dalton', 6), ('arthur munroe', 6),
    ('appleton', 6), ('ford', 6), ('lincoln', 6), ('van allister', 5),
    ('prague', 5), ('asia', 5), ('louisiana', 5), ('castro', 5), ('johansen', 5),
    ('olney court', 5), ('joseph', 5), ('charles ward', 5), ('holland', 5),
    ('atlantic', 5), ('sun', 5), ('claes van der', 5), ('pickman', 5),
    ('earl sawyer', 5), ('herbert west', 5), ('sefton', 5), ('martin beach', 5),
    ('doc', 5), ('matt', 5), ('miller', 5), ('colonies', 5), ('southward', 5),
    ('arthur wheeler', 5), ('earth', 5), ('new hampshire', 5), ('harley warren', 5),
    ('lieut klenze', 5), ('robert', 5), ('henry akeley', 5), ('norman', 5)
]

# Function to filter and categorize entities
def filter_lovecraft_entities(entities, exclude_humans=True):
    # List of human names to be excluded
    human_names = ['carter', 'willett', 'joseph curwen', 'johnny', 'ammi', 'denis', 
                   'wilbur', 'ben', 'dobson', 'george campbell', 'fed', 'van der', 'steve', 
                   'dalton', 'joe slater', 'nahum', 'davis', 'wilbur whateley', 'joe', 
                   'john', 'nova', 'romero', 'robert grandison', 'joe mazurewicz', 'warren', 
                   'joe', 'arthur jermyn', 'jermyn house', 'joe', 'arthur munroe', 'james dalton', 
                   'anderson', 'robert suydam', 'herbert west', 'sefton', 'matt', 'miller', 
                   'arthur wheeler', 'robert', 'henry akeley', 'norman']
    
    # Create a dictionary to store entities by category
    categories = {
        'Humanoid Names': [],
        'Cthulhu Mythos': [],
        'Locations & Settings': [],
        'Cosmic Entities': [],
        'Occult Entities': [],
        'Mythos-Related Concepts': []
    }
    
    # Process each entity in the list
    for entity, mentions in entities:
        # Exclude human characters
        if exclude_humans and entity.lower() in human_names:
            categories['Humanoid Names'].append((entity, mentions))
            continue
        
        # Categorize entities based on their known groupings
        if entity in ['cthulhu', 'great cthulhu', 'nyarlathotep', 'azathoth', 'shub-niggurath', 'dagon', 'yog-sothoth']:
            categories['Cthulhu Mythos'].append((entity, mentions))
        elif entity in ['arkham', 'miskatonic', 'innsmouth', 'dunwich', 'r\'lyeh', 'the dreamlands', 'the nameless city']:
            categories['Locations & Settings'].append((entity, mentions))
        elif entity in ['elder gods', 'the old ones', 'the great old ones', 'night gaunts', 'the deep ones', 'colour out of space', 'yog sothoth']:
            categories['Cosmic Entities'].append((entity, mentions))
        elif entity in ['toymakers', 'guardians of time', 'the great intelligence', 'moloch', 'hecuba', 'animus', 'archon']:
            categories['Occult Entities'].append((entity, mentions))
        else:
            categories['Mythos-Related Concepts'].append((entity, mentions))
    
    return categories

# Apply the filter
filtered_entities = filter_lovecraft_entities(entities)

# Print the result
for category, entities_list in filtered_entities.items():
    print(f"--- {category} ---")
    for entity, mentions in entities_list:
        print(f"{entity}: {mentions}")


--- Humanoid Names ---
carter: 175
willett: 99
joseph curwen: 53
johnny: 20
ammi: 19
denis: 19
wilbur: 14
ben: 14
dobson: 14
george campbell: 12
fed: 12
van der: 12
steve: 11
dalton: 11
joe slater: 10
nahum: 10
davis: 10
wilbur whateley: 10
warren: 8
joe mazurewicz: 8
john: 8
nova: 8
romero: 8
robert grandison: 8
joe: 7
arthur jermyn: 7
jermyn house: 7
robert suydam: 6
anderson: 6
james dalton: 6
arthur munroe: 6
herbert west: 5
sefton: 5
matt: 5
miller: 5
arthur wheeler: 5
robert: 5
henry akeley: 5
norman: 5
--- Cthulhu Mythos ---
nyarlathotep: 42
cthulhu: 22
great cthulhu: 18
azathoth: 18
dagon: 17
yog-sothoth: 13
shub-niggurath: 13
--- Locations & Settings ---
arkham: 74
miskatonic: 27
innsmouth: 18
dunwich: 14
the nameless city: 13
the dreamlands: 13
r'lyeh: 13
--- Cosmic Entities ---
elder gods: 17
colour out of space: 13
the great old ones: 13
the old ones: 13
the deep ones: 13
night gaunts: 13
yog sothoth: 13
--- Occult Entities ---
moloch: 14
archon: 14
the great intelligence: 